# Group by operations: split-apply-combine

**Raggruppare i valori del dataframe in base ad un campo scelto**

Ci sono 3 modi per raggruppare i valori: 

* **Suddivisione** dei dati in gruppi in base ad alcuni criteri.
* **Combinazione** dei risultati in una struttura dati.
* **Applicazione** di una funzione a ciascun gruppo in modo indipendente.

In [1]:
import pandas as pd

In [2]:
url = 'https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2015/06/phone_data.csv'
df = pd.read_csv(url)

In [3]:
df

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile
...,...,...,...,...,...,...,...
825,825,13/03/15 00:38,1.000,sms,2015-03,world,world
826,826,13/03/15 00:39,1.000,sms,2015-03,Vodafone,mobile
827,827,13/03/15 06:58,34.429,data,2015-03,data,data
828,828,14/03/15 00:13,1.000,sms,2015-03,world,world


`.groupby()` divide i dati in diversi gruppi a seconda del nome della colonna scelto. Ad esempio, l'espressione `df.groupby('month')` dividerà il nostro DataFrame corrente in base alla colonna **month**.

La funzione `.groupby()` restituisce un *oggetto GroupBy*. La variabile `.groups` dell'oggetto GroupBy è un dizionario le cui chiavi corrispondono alle colonne ed hanno come valore la lista dei campi della colonna.

In [4]:
print(type(df.groupby(['month'])))

print(df.groupby(['month']).groups.keys())

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])


Funzioni come `max()`, `min()`, `mean()`, `first()`, `last()` possono essere rapidamente applicate all'oggetto GroupBy per ottenere statistiche riassuntive per ogni gruppo.

In [5]:
# Mostra la prima riga per ogni campo raggruppato
df.groupby('month').first()

,index,date,duration,item,network,network_type
month,,,,,,
2014-11,0,15/10/14 06:58,34.429,data,data,data
2014-12,228,13/11/14 06:58,34.429,data,data,data
2015-01,381,13/12/14 06:58,34.429,data,data,data
2015-02,577,13/01/15 06:58,34.429,data,data,data
2015-03,729,12/02/15 20:15,69.000,call,landline,landline


In [6]:
# Restituisce la somma dei campi raggruppati dove possibile
df.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [7]:
# Mostra il numero dei risultati totali per ogni campo raggruppato
df.groupby('month')['date'].count()

month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

In [8]:
# Vengono filtrate solo le chiamate, per ognuna viene mostrata la duarata totale delle chiamate per ogni compagnia telegonica
df[df['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

E' possibile raggruppare anche più di una colonna, rendendo i dati più precisi

In [9]:
# Mostra il numero di chiamate, sms, e numero di trasferimento dati per ogni mese
df.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

## Statistiche multiple per gruppo
La funzione `.agg()` consente di calcolare più statistiche per gruppo in un unico calcolo.

### Applicare funzioni alle singole colonne nei gruppi
Le istruzioni per l'aggregazione sono fornite sotto forma di *dizionario* o *lista*. Le chiavi del dizionario vengono utilizzate per specificare le colonne su dove bisogna eseguire le operazioni e i valori del dizionario per specificare la funzione da eseguire.

In [10]:
operations = {
  'duration': sum,            # Restituisce la durata di ogni gruppo
  'network_type': "count",    # Restituisce il numero totali di connessioni
  'date': 'first'             # Restituisce la prima data per ogni gruppo
}

df.groupby(['month', 'item']).agg(operations)

duration  network_type            date
month   item                                         
2014-11 call  25547.000           107  15/10/14 06:58
        data    998.441            29  15/10/14 06:58
        sms      94.000            94  16/10/14 22:18
2014-12 call  13561.000            79  14/11/14 17:24
        data   1032.870            30  13/11/14 06:58
        sms      48.000            48  14/11/14 17:28
2015-01 call  17070.000            88  15/12/14 20:03
        data   1067.299            31  13/12/14 06:58
        sms      86.000            86  15/12/14 19:56
2015-02 call  14416.000            67  15/01/15 10:36
        data   1067.299            31  13/01/15 06:58
        sms      39.000            39  15/01/15 12:23
2015-03 call  21727.000            47  12/02/15 20:15
        data    998.441            29  13/02/15 06:58
        sms      25.000            25  19/02/15 18:46

### Applicazione di più funzioni alle colonne nei gruppi
Per applicare più funzioni a una singola colonna nei dati raggruppati, bisogna inserire una lista come valore della chiave dei dati.

In [11]:
# Raggruppa i dati per mese e item ed per ciascun gruppo mostra il numero di statistiche
operations = {
  'duration': [min, max, sum],         # Restituisce il valore minimo, masimo e la somma dei dati della colonna "duration"
  'network_type': "count",             # Restituisce il numero di connessioni totali
  'date': ['first', 'nunique']         # Restituisce il valore minimo, il primo valore e il numero di valori univoci (valori che non si ripetono).
}

df.groupby(['month', 'item']).agg(operations)

duration                       network_type            date  \
                  min        max        sum        count             min   
month   item                                                               
2014-11 call    1.000   1940.000  25547.000          107  01/11/14 15:13   
        data   34.429     34.429    998.441           29  01/11/14 06:58   
        sms     1.000      1.000     94.000           94  03/11/14 08:40   
2014-12 call    2.000   2120.000  13561.000           79  02/12/14 11:40   
        data   34.429     34.429   1032.870           30  01/12/14 06:58   
        sms     1.000      1.000     48.000           48  01/12/14 12:51   
2015-01 call    2.000   1859.000  17070.000           88  02/01/15 11:27   
        data   34.429     34.429   1067.299           31  01/01/15 06:58   
        sms     1.000      1.000     86.000           86  02/01/15 23:26   
2015-02 call    1.000   1863.000  14416.000           67  01/02/15 13:33   
        data   34.429     34.429   1067.299           31  01/02/15 06:58   
        sms     1.000      1.000     39.000           39  02/02/15 17:35   
2015-03 call    2.000  10528.000  21727.000           47  01/03/15 12:19   
        data   34.429     34.429    998.441           29  01/03/15 06:58   
        sms     1.000      1.000     25.000           25  02/03/15 09:19   

                                      
                       first nunique  
month   item                          
2014-11 call  15/10/14 06:58     104  
        data  15/10/14 06:58      29  
        sms   16/10/14 22:18      79  
2014-12 call  14/11/14 17:24      76  
        data  13/11/14 06:58      30  
        sms   14/11/14 17:28      41  
2015-01 call  15/12/14 20:03      84  
        data  13/12/14 06:58      31  
        sms   15/12/14 19:56      58  
2015-02 call  15/01/15 10:36      67  
        data  13/01/15 06:58      31  
        sms   15/01/15 12:23      27  
2015-03 call  12/02/15 20:15      47  
        data  13/02/15 06:58      29  
        sms   19/02/15 18:46      17